In [12]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [3]:
test=pd.read_csv('test.csv')
train=pd.read_csv('train.csv')

In [4]:
train.head(100)

,Unnamed: 0,Row ID,Incident Datetime,Incident Day of Week,Report Type Code,Intersection,Police District,Analysis Neighborhood,Supervisor District,Supervisor District 2012,Latitude,Longitude,Neighborhoods,Current Supervisor Districts,Current Police Districts,target
0,0,86814469434,2019-11-08 14:00:00,Friday,IS,MARKET ST \ POWELL ST,Tenderloin,Financial District/South Beach,6.0,3.0,37.784560,-122.407337,19.0,3.0,5.0,1
1,1,112210668395,2022-02-15 16:30:00,Tuesday,II,FRESNO ST \ GRANT AVE,Central,North Beach,3.0,3.0,37.798490,-122.407149,106.0,3.0,6.0,0
2,2,79810969452,2019-05-05 03:30:00,Sunday,IS,OSGOOD PL \ PACIFIC AVE,Central,North Beach,3.0,3.0,37.797396,-122.402887,106.0,3.0,6.0,1
3,3,127781292541,2023-05-20 18:00:00,Saturday,II,GENEVA AVE \ MOSCOW ST \ SOUTH HILL BLVD,Ingleside,Excelsior,11.0,11.0,37.713143,-122.433707,58.0,1.0,9.0,7
4,4,73242481031,2018-10-30 02:31:00,Tuesday,IS,THORNTON AVE \ BAY SHORE BLVD \ CARROLL AVE,Ingleside,Portola,9.0,10.0,37.730658,-122.403400,91.0,9.0,2.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,101252767435,2021-03-12 13:20:00,Friday,II,NIAGARA AVE \ ALEMANY BLVD,Ingleside,Outer Mission,11.0,11.0,37.716970,-122.443431,80.0,1.0,9.0,15
96,96,80186627391,2019-05-17 10:05:00,Friday,II,GEARY ST \ POWELL ST,Central,Financial District/South Beach,3.0,3.0,37.787359,-122.408227,19.0,3.0,6.0,18
97,97,90424370602,2020-02-17 05:47:00,Monday,II,19TH ST \ ARKANSAS ST,Bayview,Potrero Hill,10.0,10.0,37.761184,-122.398338,54.0,9.0,2.0,5
98,98,110925992531,2022-01-01 23:00:00,Saturday,II,VICENTE ST \ 39TH AVE,Taraval,Sunset/Parkside,4.0,4.0,37.738379,-122.496855,39.0,7.0,10.0,7


In [ ]:
# Проверка на наличие необходимых колонок
if 'Latitude' not in train.columns or 'Longitude' not in train.columns or 'Police District' not in train.columns:
    raise ValueError("Датасет должен содержать колонки 'Latitude', 'Longitude' и 'Police District'.")

# Удаление строк с NaN в колонках 'Latitude', 'Longitude' и 'Police District'
train = train.dropna(subset=['Latitude', 'Longitude', 'Police District'])

# Проверка на наличие данных после удаления NaN
if train.empty:
    raise ValueError("После удаления NaN в колонках 'Latitude', 'Longitude' и 'Police District' датасет пуст.")

# Создание карты с начальной точкой
m = folium.Map(location=[train['Latitude'].mean(), train['Longitude'].mean()], zoom_start=6)

# Создание объекта кластеризации
marker_cluster = MarkerCluster().add_to(m)

# Добавление маркеров на карту с кластеризацией
for index, row in train.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Index: {index}<br>Police District: {row['Police District']}",
        icon=folium.Icon(color='blue')
    ).add_to(marker_cluster)

# Сохранение карты в HTML файл
m.save('map.html')

m